In [13]:
from simulated_bifurcation import Ising
from tqdm import tqdm
import matplotlib.pyplot as plt
import pulp as pl
import numpy as np
import time
import json

N_range = [10]
alpha_range = [10]

def generate_J_h(N_range,alpha_range):
    for N in N_range:
        for alpha in alpha_range:
            J = np.random.randint(-100,100,size=(N*alpha, N*alpha))/(10**6)
            J = (J + J.T)/2
            J = J - np.diag(np.diag(J))
            h = np.random.randint(-100,100,size=N*alpha)/(10**4)

            with open('test_cbc/test_CBC_J_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, J)

            with open('test_cbc/test_CBC_h_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, h)

            with open('test_cbc/test_SB_J_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, J/4)

            with open('test_cbc/test_SB_h_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, h/2 - sum([J[:,i]/8 for i in range(N*alpha)]) - sum([J[i,:]/8 for i in range(N*alpha)]))

generate_J_h(N_range,alpha_range)

In [3]:
def create_optimize_gurobi(J,h,J_SB,h_SB,max_seconds=-1):
    time_1 = time.time()
    size = J.shape[0]

    model = pl.LpProblem("Linear Optimization", pl.LpMinimize)
    x = pl.LpVariable.dicts('x', list(range(size)), lowBound=0, upBound=1, cat='Integer')
    z = pl.LpVariable.dicts('z', [(i,j) for i in range(size) for j in range(size)], lowBound=0, upBound=1, cat='Integer')

    for i in range(size):
        for j in range(size):
            model += z[i,j] <= x[i]
            model += z[i,j] <= x[j]
            model += z[i,j] >= x[i] + x[j] -1

    model += pl.lpSum([(-1/2)*J[i][j]*z[i,j] for i in range(size) for j in range(size)]) + pl.lpSum([x[i]*h[i] for i in range(size)])

    if max_seconds == -1:
        model.solve(pl.PULP_CBC_CMD(msg=0, timeLimit=60*10))
    else:
        model.solve(pl.PULP_CBC_CMD(msg=0, timeLimit=max_seconds))

    spins = [1 if x[i].varValue >= 0.99 else -1 for i in range(size)]
    energy = (-1/2)*sum([sum([J_SB[i,j]*spins[j] for j in range(size)])*spins[i] for i in range(size)]) + sum([h_SB[i]*spins[i] for i in range(size)])[0]

    processing_time = time.time() - time_1
    return (processing_time,np.array(spins),energy)

In [4]:
def create_optimize_SB(J,h):
    time_1 = time.time()

    model = Ising(J,h)
    model.optimize(convergence_threshold = 35,
        sampling_period = 60,
        time_step = 0.01, 
        symplectic_parameter = 2,
        agents = 5,
        ballistic=False,
        heated=True,
        final_pressure=1.,
        pressure_slope=0.2)

    processing_time = time.time() - time_1
    return(processing_time, model.ground_state, model.energy)

In [5]:
def test_protocol(N_range, alpha_range):
    energy_precision = {}
    proc_times = {}
    energy_precision_stop = {}

    for N in tqdm(N_range):
        energy_precision[N] = {}
        proc_times[N] = {}
        energy_precision_stop[N] = {}
        for alpha in alpha_range:
            with open('test_cbc/test_SB_J_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_SB_J = np.load(f)

            with open('test_cbc/test_SB_h_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_SB_h = np.expand_dims(np.load(f),axis=1)

            with open('test_cbc/test_CBC_J_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_CBC_J = np.load(f)

            with open('test_cbc/test_CBC_h_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_CBC_h = np.load(f)

            processing_time_sb, _, energy_sb = create_optimize_SB(test_SB_J,test_SB_h)
            processing_time_cbc, _, energy_cbc = create_optimize_gurobi(test_CBC_J,test_CBC_h,test_SB_J,test_SB_h)
            _, _, energy_cbc_stopped = create_optimize_gurobi(test_CBC_J,test_CBC_h,test_SB_J,test_SB_h,processing_time_sb)

            energy_precision[N][alpha] = round((energy_sb - energy_cbc)/abs(energy_cbc),6)
            proc_times[N][alpha] = round(processing_time_cbc/processing_time_sb,2)
            energy_precision_stop[N][alpha] = round((energy_cbc_stopped - energy_sb)/abs(energy_sb),6)

    return energy_precision,proc_times,energy_precision_stop

In [8]:
precisions_dict, times_dict, precisions_early_stopping_dict = test_protocol(N_range,alpha_range)

with open('test_cbc/precisions.json', 'w') as json_file:
    json.dump(precisions_dict,json_file)

with open('test_cbc/precisions_early_stopping.json', 'w') as json_file:
    json.dump(precisions_early_stopping_dict,json_file)

with open('test_cbc/times.json', 'w') as json_file:
    json.dump(times_dict,json_file)

  0%|          | 0/1 [00:00<?, ?it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Bifurcated spins 0/5 Spins bifurcated in 0.177 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved


100%|██████████| 1/1 [00:05<00:00,  5.57s/it]


In [9]:
with open('test_cbc/precisions.json', 'r') as json_file:
    precisions_dict = json.load(json_file)

with open('test_cbc/precisions_early_stopping.json', 'r') as json_file:
    precisions_early_stopping_dict = json.load(json_file)

with open('test_cbc/times.json', 'r') as json_file:
    times_dict = json.load(json_file)

In [10]:
times_dict

{'10': {'10': 8.15}}

In [11]:
precisions_early_stopping_dict

{'10': {'10': 0.0}}

In [12]:
precisions_dict

{'10': {'10': 0.0}}